# 16.7 Async Youtuber Crew

아래의 task 가 순서대로 실행된다.

- brainstorm_task
  - 5 가지 아이디어를 제공
- selection_task
  - 5 가지의 아이디어에서 하나를 선택
- research_task
  - 선택한 아이디어에 대한 검색
- inspiration_task
  - 우리가 작업 중인 영상 아이디어와 비슷하지만 다른산업에서 제작된 비디오나 기사를 찾아보기
- competitors_task
  - 경쟁사 분석과 그것들과 차별화 될 수 있는 아이디어를 제시해라
- script_task
  - script 작성


## MEMO

crew ai 에서 async_execution=True 르 적으면 비동기로 task 를 실행한다.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [4]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    allow_delegation=False,
    max_iter=10,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    allow_delegation=False,
    max_iter=10,
    verbose=True,
)

In [8]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="outputs/ideas_task.md",
    human_input=True, # 사용자 입력을 받기 위해 True로 설정
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,  # 사용자 입력을 받기 위해 True로 설정
    context=[ # dependency: 이전 작업의 결과를 사용하기 위해 context에 추가
        brainstorm_task,
    ],
    output_file="outputs/selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True, # 비동기 처리를 위해 True로 설정
    context=[ # dependency: 이전 작업의 결과를 사용하기 위해 context에 추가(ex. serach_task 는 selection_task의 결과를 사용)
        selection_task,
    ],
    output_file="outputs/research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True, # 비동기 처리를 위해 True로 설정
    context=[ # dependency: 이전 작업의 결과를 사용하기 위해 context에 추가
        selection_task,
    ],
    output_file="outputs/competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True, # 비동기 처리를 위해 True로 설정
    context=[ # dependency: 이전 작업의 결과를 사용하기 위해 context에 추가
        selection_task,
    ],
    output_file="outputs/inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[ # dependency: 이전 작업의 결과를 사용하기 위해 context에 추가
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="outputs/script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

result

2025-01-15 18:46:15,815 - 8501035584 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer.

Final Answer:

1. **"The Ultimate Hot Sauce Challenge: Spiciest Foods Around the World"**
   - **Angle**: Create an engaging and entertaining video by challenging a diverse group of people to taste-test the spiciest hot sauces from different countries. Capture their reactions and provide interesting facts about each sauce's origin and ingredients. This video will appeal to food enthusiasts and thrill-seekers, creating a sense of adventure and cultural exploration.

2. **"DIY Hot Sauce: How to Create Your Signature Flavor"**
   - **Angle**: Guide viewers through the process of making their own hot sauce at home, encouraging creativity and personalization. Include tips on selecting ingredients, balancing flavors, and preserving the sauce. Feature guest appearan

'**Title:** Hot Ones Style: Interviewing Celebrities Over Spicy Wings\n\n**Introduction:**\n- [Opening shot of the host with a platter of spicy wings]\n- **Host:** "Welcome to [Your Show Name], the show with hot questions and even hotter wings. Today, we\'re sitting down with [Celebrity Guest], known for [a brief introduction of the guest’s achievements]. Get ready as we dive into the heat and uncover some fiery truths!"\n\n**Segment 1: Warm-Up**\n- [Camera pans to the array of hot sauces]\n- **Host:** "We have a range of sauces today, starting mild and ramping up to the infamous [name of the hottest sauce]. Let\'s kick things off with our first wing."\n- [Guest takes a bite of the first wing]\n- **Host:** "So, [Guest], let\'s start with something easy. I heard you\'re a big fan of [Guest\'s known interest]. Can you tell us more about that?"\n\n**Segment 2: Deep Dive**\n- [As wings get hotter, questions become more personal or insightful]\n- **Host:** "Alright, onto our second wing. Th